In [1]:
from LSTM_enc_dec import *
from data_preprocessing import *

# Specify the model number of the model to be loaded
model_num = 657521

saved_model = torch.load(f"./trained_models/model_{model_num}.pt")

# Load the hyperparameters from the saved model
params = saved_model["hyperparameters"]

hidden_size = params["hidden_size"]
num_layers = params["num_layers"]
learning_rate = params["learning_rate"]
num_epochs = params["num_epochs"]
input_window = params["input_window"]
output_window = params["output_window"]
batch_size = params["batch_size"]
training_prediction = params["training_prediction"]
teacher_forcing_ratio = params["teacher_forcing_ratio"]
dynamic_tf = params["dynamic_tf"]
shuffle = params["shuffle"]
loss_type = params["loss_type"]
wind_farm = params["wind_farm"]

Saved model : {'hyperparameters': {'hidden_size': 256, 'num_layers': 3, 'learning_rate': 0.001, 'num_epochs': 3, 'input_window': 12, 'output_window': 12, 'batch_size': 64, 'training_prediction': 'mixed_teacher_forcing', 'teacher_forcing_ratio': 0.6, 'dynamic_tf': True, 'loss': [0.29396964676632753, 0.23632628086772633, 0.22041264623857854], 'loss_test': [0.4090023660586655, 0.39226080129848667, 0.3849992515302143], 'loss_type': 'L1', 'shuffle': True, 'wind_farm': 'brazil'}, 'model_state_dict': OrderedDict([('encoder.lstm.weight_ih_l0', tensor([[-0.0163,  0.0112,  0.0556,  ..., -0.0650,  0.0155,  0.0054],
        [-0.0142,  0.0721,  0.0405,  ...,  0.0610,  0.0041, -0.0571],
        [-0.0014,  0.0431, -0.0211,  ..., -0.0036, -0.0545,  0.0571],
        ...,
        [-0.0480,  0.0119, -0.0108,  ..., -0.0449, -0.0559,  0.0330],
        [-0.0791, -0.0386,  0.0319,  ..., -0.0569, -0.0386,  0.0319],
        [ 0.0602,  0.0581,  0.0252,  ...,  0.0895,  0.0238,  0.0274]])), ('encoder.lstm.weight_

In [2]:
# Load the new data for transfer learning
data = pd.read_csv('./preprocessed_data/filtered_dataset_britain2_time_lag_corr.csv')
data = np.array(data).T
data = torch.from_numpy(data)

# Calculate the mean and standard deviation along the feature dimension
data = normalize_data(data)

# Split the data into training and testing sets
index_train = int(0.8 * len(data[0, :]))
data_train = data[:, :index_train]
data_test = data[:, index_train:]

# Specify the number of features and the stride for timeseries data
num_features = 11
stride = 1

input_data, target_data = dataloader_seq2seq_feat(data_train,
                                                  input_window=input_window,
                                                  output_window=output_window,
                                                  stride=stride,
                                                  num_features=num_features)

input_data_test, target_data_test = dataloader_seq2seq_feat(data_test,
                                                            input_window=input_window,
                                                            output_window=output_window,
                                                            stride=stride,
                                                            num_features=num_features)

In [3]:
# Specify the device to be used for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# convert windowed data from np.array to PyTorch tensor
data_train, target, data_test, target_test = numpy_to_torch(input_data, target_data, input_data_test, target_data_test)

# Initialize the model
model = LSTM_Sequence_Prediction(input_size = data_train.shape[2], hidden_size = hidden_size, num_layers=num_layers)

# Load the saved model
model.load_state_dict(saved_model["model_state_dict"])
model.to(device)

# Load the saved optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer.load_state_dict(saved_model['optimizer_state_dict'])

In [4]:
# Train the model
loss, loss_test = model.train_model(data_train,
                                    target,
                                    data_test,
                                    target_test,
                                    num_epochs,
                                    input_window,
                                    output_window,
                                    batch_size,
                                    training_prediction,
                                    teacher_forcing_ratio,
                                    learning_rate,
                                    dynamic_tf,
                                    loss_type,
                                    optimizer)

# Append the model number to indicate transfer learning
rand_identifier = str(model_num) + "tl"

# Save the model and hyperparameters to a file
parameters = {
        'hidden_size': hidden_size,
        "num_layers": num_layers,
        'learning_rate': learning_rate,
        'num_epochs': num_epochs,
        "input_window": input_window,
        "output_window": output_window,
        "batch_size": batch_size,
        "training_prediction": training_prediction,
        "teacher_forcing_ratio": teacher_forcing_ratio,
        "dynamic_tf": dynamic_tf,
        "loss": loss.tolist(),
        "loss_test": loss_test.tolist(),
        "loss_type": loss_type,
        "shuffle": shuffle,
        "wind_farm": wind_farm,
    }

torch.save({'hyperparameters': parameters,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            },
	    f'./trained_models/model_{rand_identifier}.pt')

cpu
Number of batches : 658
Number of batches test : 164


  0%|          | 0/3 [00:03<?, ?it/s]

KeyboardInterrupt

